#### G1收集器
1. G1的特点
    1. 划分Region :   
       G1收集器不再对堆内存分代, 而是划分为多个大小相同的region. 划分方式有有2种 : 
        1. 指定每个region的大小. 通过`-XX:G1HeapRegionSize`, 大小区间在1M和32M之间, 但要是2的幂
        2. 不指定参数时, 默认把堆内存等分成2048份
    2. 为Region标记  
        G1对每个Region进行标记为4中类型: `Eden, Survivor, Old, Hunmongous`. Humongous用于存放大小超过region大小一半的大对象. 如果1个Region存不下这个大对象, G1会找来连续的H区域装下这个对象, 为了找到连续的H区域可能会不得不切换成serialOldGC进行全堆扫描(eden, survivor, old, hunmongous)  
    3. 属于"标记-整理"算法
    4. 用户客户可以指定gc停顿时间
        

2. G1的收集模式
    1. YoungGC  
       正在分配非巨型对象(在eden去分配的对象)时,  如果发现Eden区满了则会触发YoungGC. 每次YoungGC都会回收Eden region和Survivor region, 然后一部分对象将进入old region. YoungGC过程如下  
    2. MixedGC
       当老年代(old region)消耗殆尽时, 会触发混合gc(MixedGC), 会回收整个young region和一部分的old region.而选择哪部分old region收集是通过用户指定的参数`-XX:MaxGCPauseMillis`推导而来. 改参数时用户设置的gc停顿时间, 选择那些预估gc时间和用户设置时间相近的old region进行gc


3. G1垃圾回收的步骤: 
    1. 初始标记
    2. 并发标记
    3. 最终标记
    4. 筛选清除

### 二. GC日志
1. `-XX:+PrintGCDetails`  
   该参数告诉虚拟机在进行gc时打印内存回收日志, 并在进程退出时打印各个区的分配情况. 配合该参数的2个额外功能 : 
    1. `-XX:+PrintGCDateStamps` : 打印GC日志时带上时间
    2. `-Xloggc:<file_name>` : 将gc日志打印到文件中
2. GC日志格式  
   如下代码产生了如下的gc日志:
    1. 2019-09-24T00:03:18.695+0800 : gc发生时间
    2. GC (Allocation Failure) : gc类型
    3. PSYoungGen: 8113K->824K(9216K) : 青年代gc后由8113K减小到824K, 共9216K
    4. 8113K->6976K(19456K), 0.0037173 secs : 总堆内存大小变化和gc耗时
   
```java
/** GC日志 */
2019-09-24T00:03:18.695+0800: [GC (Allocation Failure) [PSYoungGen: 8113K->824K(9216K)] 8113K->6976K(19456K), 0.0037173 secs] [Times: user=0.00 sys=0.00, real=0.00 secs] 
2019-09-24T00:03:18.699+0800: [Full GC (Ergonomics) [PSYoungGen: 824K->0K(9216K)] [ParOldGen: 6152K->6764K(10240K)] 6976K->6764K(19456K), [Metaspace: 3222K->3222K(1056768K)], 0.0053386 secs] [Times: user=0.00 sys=0.00, real=0.01 secs] 

/** 进程退出时打印的各个区分配情况 */
Heap
 PSYoungGen      total 9216K, used 2266K [0x00000000ff600000, 0x0000000100000000, 0x0000000100000000)
  eden space 8192K, 27% used [0x00000000ff600000,0x00000000ff836b50,0x00000000ffe00000)
  from space 1024K, 0% used [0x00000000ffe00000,0x00000000ffe00000,0x00000000fff00000)
  to   space 1024K, 0% used [0x00000000fff00000,0x00000000fff00000,0x0000000100000000)
 ParOldGen       total 10240K, used 6764K [0x00000000fec00000, 0x00000000ff600000, 0x00000000ff600000)
  object space 10240K, 66% used [0x00000000fec00000,0x00000000ff29b0e8,0x00000000ff600000)
 Metaspace       used 3228K, capacity 4496K, committed 4864K, reserved 1056768K
  class space    used 350K, capacity 388K, committed 512K, reserved 1048576K

```

### 三. 对象的内存分配策略
Java自动内存管理分为2部分: 自动回收内存(gc)和自动分配内存(对象内存分配); 前面介绍了自动回收内存, 下面介绍自动发分配内存的几点
#### 1. 优先在Eden区分配, minor gc时Servivor存不下的对象直接进入老年代

#### 2. 大对象直接进入老年代 (-XX:PretenureSizeThreshold)
1. 大对象是指占用大量连续的内存空间的java对象, 比如大数组类型对象. 这种大对象当超过-XX:PretenureSizeThreshold配置时会直接在老年代生成. 原因是避免minor gc时反复从eden, survivor0, survivor1之间相互拷贝  
2. 最影响性能的就是那些"朝生夕灭"的大对象  

#### 3. Serial中的对象何时进入老年代?
1. Serial中高年龄对象进入老年代 (-XX:MaxTenuringThresholdSize)
2. minor gc时Servivor存不下的对象直接进入老年代
2. 当Survivor对象中大于一半的对象有相同的age时, 大于等于该age的对象直接进入老年代

#### 4. 老年代的空间担保, 担保失败会进入full gc
除了老年代gc会导致进入full gc外, 青年代对象的产生也可能导致full gc的产生. 这是因为minor gc需要老年代提供担保:   
1. 何时担保?  
当发生minor gc时, 如果发现survivor无法容纳对象继续进入, 则这些对象会直接进入老年代, 而进入成功的前提是老年代的剩余空间大于这些对象的总和, 因此需要老年代在此时做出担保 


2. 老年代需要担保多少空间?  
因为Survivor并不知道一共有多少对象无法容纳, 因此只好取每次晋升到老年代对象容量的平均值作为一个经验值.   
    * 如果老年代剩余空间 > 平均值, 担保成功
    * 反之, 担保失败, 担保失败会触发full gc  
    
 JDK6U24后, 默认只要`老年代剩余最大连续空间`大于`新生代对象总大小`or`每次晋升到老年代的平均大小`则担保成功, 进行minorgc, 否则进行full gc
